## Vessel segmentation using a U-Net model
This notebook demonstrates the use of the vseg vessel segmentation network included in the toolkit. The model is trained on LSFM data.

In [ ]:
# Import the necessary libraries
import os
import shutil

import numpy as np
from liom_toolkit.segmentation.vseg.model import VsegModel
from liom_toolkit.segmentation.vseg.predict_one import predict_one
from liom_toolkit.utils import extract_zarr_to_png, generate_label_color_dict_mask, save_label_to_zarr
from tqdm.auto import tqdm

### Prepare the zarr volume
The individual slices need to be extracted from the zarr volume and saved as individual png files. The following code demonstrates how to do this.

In [ ]:
zarr_file = ""
png_dir = ""
extract_zarr_to_png(zarr_file, png_dir)

### Run the model
The following code downloads the pre-trained model and runs it for the entire folder

In [ ]:
# Set the device, 'cuda' if the machine has an Nvidia GPU and CUDA installed, otherwise 'cpu'
device = "cuda"
# Model load, the pretrained options downloads the most recent from wandb
model = VsegModel(pretrained=True, device=device)

In [ ]:
dir_path = png_dir
output_path = ""
normalization = True
patching = False
stride = None
width = None

In [ ]:
# Run the model
vessel_stack = []
for images in tqdm(os.listdir(dir_path), desc="Processing images"):
    image_path = os.path.join(dir_path, images)
    prediction = predict_one(model=model, img_path=image_path, save_path=output_path,
                             norm=normalization, dev=device, patching=patching, stride=stride, width=width)
    vessel_stack.append(prediction)
volume = np.stack(vessel_stack)
volume = np.transpose(volume, (2, 1, 0))
volume = volume.astype("uint8")

### Reassemble the slices to save as zarr
Next, we need to save the files back to a zarr volume. The following code demonstrates how to do this.

In [ ]:
# Make the volume binary
volume[volume > 0] = 1
# Transpose the volume to match the zarr format
volume = np.transpose(volume, (2, 1, 0))

# Save the volume to zarr
colour_dict = generate_label_color_dict_mask()
save_label_to_zarr(volume, zarr_file, colour_dict, "vessels", resolution_level=0)

### Cleanup
Finally, we can clean up the png files that were created.

In [ ]:
shutil.rmtree(png_dir)